In [37]:
from bs4 import BeautifulSoup
import requests

In [38]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
# Retrieve page with the requests module
response = requests.get(url)
response

<Response [200]>

In [43]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [40]:
news_title = soup.find('div', class_="content_title").text.strip()
print(news_title)

NASA Readies Perseverance Mars Rover's Earthly Twin


In [44]:
news_teaser_title = soup.find('div', class_="rollover_description_inner").text.strip()
print(news_teaser_title)

Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.
